In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from pytorch_transformers import AdamW
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\603766\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from bert_serving.client import BertClient

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
df = pd.read_csv('../Data/Product_Feelings.csv')

In [5]:

df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [48]:
sentences = df.tweet_text.values
new_sent=[]
for i in sentences:
    new_sent.append(str(i))
sentences = [sentence + " [SEP] [CLS]" for sentence in new_sent]
labels = df.is_there_an_emotion_directed_at_a_brand_or_product.values

In [49]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange

In [50]:
MAX_LEN = 40
bs =64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [51]:
#torch.cuda.get_device_name(0) 
tokenizer_1 = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts_1 = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts_1[0])

['.', '@', 'wesley', '##8', '##3', 'i', 'have', 'a', '3', '##g', 'iphone', '.', 'after', '3', 'hr', '##s', 't', '##wee', '##ting', 'at', '#', 'rise', '_', 'austin', ',', 'it', 'was', 'dead', '!', 'i', 'need', 'to', 'upgrade', '.', 'plug', '##in', 'stations', 'at', '#', 's', '##x', '##sw', '.', '[SEP]', '[CLS]']


In [30]:
tags_vals = list(set(df["is_there_an_emotion_directed_at_a_brand_or_product"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [31]:
input_ids_1 = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_1],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [32]:
input_ids_1 = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [33]:
tokenizer_2 = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_texts_2 = [tokenizer.tokenize(sent) for sent in sentences]

In [ ]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

len(svd_model.components_)

In [34]:
print ("Tokenize the first sentence:")
print (tokenized_texts_2[0])

Tokenize the first sentence:
['.', '@', 'wesley', '##8', '##3', 'i', 'have', 'a', '3', '##g', 'iphone', '.', 'after', '3', 'hr', '##s', 't', '##wee', '##ting', 'at', '#', 'rise', '_', 'austin', ',', 'it', 'was', 'dead', '!', 'i', 'need', 'to', 'upgrade', '.', 'plug', '##in', 'stations', 'at', '#', 's', '##x', '##sw', '.', '[SEP]', '[CLS]']


In [35]:
MAX_LEN = 128

In [36]:
input_ids_2 = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_2]

In [37]:
input_ids_2 = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [38]:
X_train_1, X_test_1,y_train_1,y_test_1 = train_test_split(input_ids_1,labels,test_size=.2)

In [39]:
X_train_2, X_test_2,y_train_2,y_test_2 = train_test_split(input_ids_2,labels,test_size=.2)

In [40]:
from sklearn.ensemble import RandomForestClassifier
inst_rf=RandomForestClassifier()
fit_rf=inst_rf.fit(X_train_1,y_train_1)
y_pred_1=fit_rf.predict(X_test_1)
from sklearn.metrics import accuracy_score
accuracy_score(y_pred_1,y_test_1)

C:\Users\603766\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5536008796041781

In [41]:
inst_rf=RandomForestClassifier()
fit_rf=inst_rf.fit(X_train_2,y_train_2)
y_pred_2=fit_rf.predict(X_test_2)
from sklearn.metrics import accuracy_score
accuracy_score(y_pred_2,y_test_2)

C:\Users\603766\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5514018691588785

In [42]:
from transformers import BertModel
sentences = df.tweet_text.values
new_sent=[]
for i in sentences:
    new_sent.append(str(i))
sentences = [sentence + " [SEP] [CLS]" for sentence in new_sent]
labels = df.is_there_an_emotion_directed_at_a_brand_or_product.values


In [13]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
data = new_sent

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I '
 'need to upgrade. Plugin stations at #SXSW.']


In [15]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['have', 'iphone', 'after', 'hrs', 'tweeting', 'at', 'rise_austin', 'it', 'was', 'dead', 'need', 'to', 'upgrade', 'plugin', 'stations', 'at', 'sxsw']]


In [16]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['have', 'iphone', 'after', 'hrs', 'tweeting', 'at', 'rise_austin', 'it', 'was', 'dead', 'need', 'to', 'upgrade', 'plugin', 'stations', 'at', 'sxsw']


In [32]:
import spacy
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['tweet', 'dead', 'need', 'upgrade', 'plugin', 'station']]


In [33]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [39]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('dead', 1),
  ('need', 1),
  ('plugin', 1),
  ('station', 1),
  ('tweet', 1),
  ('upgrade', 1)]]

In [35]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [36]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.183*"new" + 0.177*"launch" + 0.157*"circle" + 0.146*"today" + '
  '0.085*"social" + 0.045*"major" + 0.035*"possibly" + 0.020*"network" + '
  '0.018*"lot" + 0.015*"medium"'),
 (1,
  '0.331*"be" + 0.185*"need" + 0.141*"talk" + 0.075*"tweet" + 0.048*"sure" + '
  '0.023*"stand" + 0.015*"upgrade" + 0.010*"station" + 0.000*"line" + '
  '0.000*"guy"'),
 (2,
  '0.928*"link" + 0.019*"attend" + 0.000*"line" + 0.000*"temporary" + '
  '0.000*"set" + 0.000*"guy" + 0.000*"location" + 0.000*"other" + 0.000*"big" '
  '+ 0.000*"live"'),
 (3,
  '0.245*"say" + 0.175*"user" + 0.133*"google" + 0.080*"must" + 0.058*"smart" '
  '+ 0.030*"idea" + 0.021*"blog" + 0.012*"effort" + 0.011*"system" + '
  '0.010*"allow"'),
 (4,
  '0.317*"see" + 0.219*"day" + 0.065*"enjoy" + 0.055*"help" + 0.047*"mean" + '
  '0.030*"change" + 0.026*"connect" + 0.025*"else" + 0.013*"stock" + '
  '0.013*"gear"'),
 (5,
  '0.551*"app" + 0.188*"amp" + 0.115*"year" + 0.021*"wonder" + 0.000*"line" + '
  '0.000*"session" + 0.000*"b

In [29]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -15.478334944472223

Coherence Score:  0.4654683200046127


In [31]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

IndexError: invalid index to scalar variable.